# Reclassify user/contact organizations

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
object_id = 'Contact'
external_id = 'nanoHUB_user_ID__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"shit9927289sSYTkLiGvyK2UrazHFmjFUGU",    
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",


    "username": "wang159-4j1v@force.com",
    "password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [2]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import datetime

## Obtain user org classifications from Salesforce

In [3]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# get Salesforce ID for organizations
contact_org_df = db_s.query_data('SELECT Id, nanoHUB_user_ID__c, Email, Organization_composite__c,\
Organization_email_derived__c, Organization_Conflict__c, \
Organization__c FROM Contact Order by nanoHUB_user_ID__c DESC limit 5000') # limit 5000

#Organization_citation_derived__c

# display
contact_org_df.head(3)

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000PVjhGAAT
{"id":"7505w00000PVjhGAAT","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-09-02T16:51:56.000+0000","systemModstamp":"2020-09-02T16:51:57.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000PVjhGAAT","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-09-02T16:51:56.000+0000","systemModstamp":"2020-09-02T16:52:00.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000PVjhGAAT","operation":"query","object":"Contact","createdById":"0055w0000

,Email,Id,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
0,jack@test.com,0035w000033yK8wAAE,False,NaN,NaN,NaN,NaN
1,asdf1234@invalidaddress5.org,0035w00003DA2NpAAL,False,NaN,NaN,NaN,NaN
2,tonya@sample.com,0035w000033yK8yAAE,False,NaN,NaN,NaN,NaN


In [4]:
# pull all the organizations from SF to get their unique tokens
all_org_df = db_s.query_data('SELECT Id, Name, Domain__c FROM Organization__c')

display(all_org_df.head(2))
print(all_org_df.shape)

[Success] Bulk job creation successful. Job ID = 7505w00000PVk0pAAD
{"id":"7505w00000PVk0pAAD","operation":"query","object":"organization__c","createdById":"0055w00000DM5bOAAT","createdDate":"2020-09-02T16:52:45.000+0000","systemModstamp":"2020-09-02T16:52:47.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":6274,"retries":0,"totalProcessingTime":781}
[Success] Bulk job completed successfully.


,Domain__c,Id,Name
0,upenn.edu,a0r5w00000V42c0AAB,university of pennsylvania
1,fer.hr,a0r5w00000V42c1AAB,fer


(6274, 3)


In [5]:
## extract the domain from contact_org_df
user_domains = contact_org_df['Email'].fillna(' ').apply(lambda x: x.split('@')[-1])
display(user_domains.head(2))

0               test.com
1    invalidaddress5.org
Name: Email, dtype: object

In [6]:
## if contact domain is a commercial email domain, then find the index and empty out the entry 
import commercial_domains as cdomains
user_domains = user_domains.apply(lambda x: ' ' if x in cdomains.domains else x)
display(user_domains.head(2))

0               test.com
1    invalidaddress5.org
Name: Email, dtype: object

In [7]:
## finding indexes in contact_org_df and zero out Organization_email_derived__c
indices = [i for i, x in enumerate(user_domains.to_list()) if x == ' ']

contact_org_df['Organization_email_derived__c'][indices] = ' '

## list comprehension method
display(contact_org_df.head(5))


C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Email,Id,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
0,jack@test.com,0035w000033yK8wAAE,False,NaN,NaN,NaN,NaN
1,asdf1234@invalidaddress5.org,0035w00003DA2NpAAL,False,NaN,NaN,NaN,NaN
2,tonya@sample.com,0035w000033yK8yAAE,False,NaN,NaN,NaN,NaN
3,alex@bco.com,0035w000033yK8zAAE,False,NaN,NaN,NaN,NaN
4,ed@getoutdoors.com,0035w000033yK90AAE,False,NaN,NaN,NaN,NaN


In [8]:
## match/find all_org_df ID values
nh_orgs = contact_org_df['Organization__c'].fillna(' ').apply(lambda x: x.lower())
sf_org_strings = all_org_df['Name'].fillna(' ').apply(lambda x: x.lower()).to_list()
sf_org_ids = all_org_df['Id'].fillna(' ')

In [9]:
# continued from above snippet
indices = [i for i, x in enumerate(nh_orgs.to_list()) if x != ' ']

#check if organization already has a SF entry
for i in indices:
    if nh_orgs[i] in sf_org_strings: #if exact match
        t_index = sf_org_strings.index(nh_orgs[i]) #pull exact id
        nh_orgs[i] = sf_org_ids[t_index]
    else:
        nh_orgs[i] = ' '
        
#    else:
#        #calculate the damerau levenshtein distance
#        min_dl_dist,org_index = dl_dist(nh_orgs[i],sf_org_strings)
#        if min_dl_dist <= 4: #fit the org to the org_index 
#            t_index = org_index
#            nh_orgs[i] = sf_org_ids[t_index]
        ## create new entry is involved... create a new DF, populate with new org entries, send to SF
        ## pull SF assigned new org ID back, then repopulate the contacts SF DF with new org ID only for those new orgs 
        #else: #create new entry
        #    t_index = org_index

In [10]:
display(nh_orgs.head(3))

0     
1     
2     
Name: Organization__c, dtype: object

In [11]:
## Compare and readjust the organization_email_derived__c based on email domains
email_orgs = contact_org_df['Organization_email_derived__c']
emails = contact_org_df['Email'].fillna(' ').apply(lambda x: x.split('@')[-1])

In [12]:
## all sf domains
sf_domains = all_org_df['Domain__c'].to_list()

In [13]:
display(contact_org_df.head(2))
display(contact_org_df.tail(2))

,Email,Id,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
0,jack@test.com,0035w000033yK8wAAE,False,NaN,NaN,NaN,NaN
1,asdf1234@invalidaddress5.org,0035w00003DA2NpAAL,False,NaN,NaN,NaN,NaN


,Email,Id,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
224744,support@nanohub.org,0035w000031Vsp2AAC,False,purdue university,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR,1683.0
224745,nkissebe@gmail.com,0035w000031Vsp1AAC,False,NaN,NaN,,998.0


In [14]:
## verify that the email_org entry corresponds to the correct organizational domain
for i,j in enumerate(emails.to_list()):
    ## apply fixes for mail.usf.edu to usf.edu - i.e., use the simplest derivative of the domains    
    j_test = j.split('.')
    if len(j_test) >= 3:
        j = '.'.join(j_test[-2:])
    
    if j in sf_domains:
        # pull the corresponding org Id
        j_org = all_org_df['Id'][sf_domains.index(j)]
        contact_org_df['Organization_email_derived__c'][i] = j_org
        
display(contact_org_df.head(2))
display(contact_org_df.tail(2))

C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Email,Id,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
0,jack@test.com,0035w000033yK8wAAE,False,NaN,NaN,NaN,NaN
1,asdf1234@invalidaddress5.org,0035w00003DA2NpAAL,False,NaN,NaN,NaN,NaN


,Email,Id,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
224744,support@nanohub.org,0035w000031Vsp2AAC,False,purdue university,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR,1683.0
224745,nkissebe@gmail.com,0035w000031Vsp1AAC,False,NaN,NaN,,998.0


In [15]:
## Compare the SF indexes of NH determined organization against that of Organization_email_derived__c
email_orgs = contact_org_df['Organization_email_derived__c']

for i,j in enumerate(nh_orgs.to_list()):
    if j != ' ' and email_orgs[i] == ' ':
        # nanohub org overwrites all other org flags
        contact_org_df['Organization_composite__c'][i] = j
    elif j != ' ' and j != email_orgs[i]:
        contact_org_df['Organization_Conflict__c'][i] = True
        contact_org_df['Organization_composite__c'][i] = j

    elif j != ' ' and j == email_orgs[i]:
        contact_org_df['Organization_Conflict__c'][i] = False
        contact_org_df['Organization_composite__c'][i] = j

        
display(contact_org_df.head(4))
display(contact_org_df.tail(4))

C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\henry\Anaconda3\envs\tf\lib\site-pack

,Email,Id,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
0,jack@test.com,0035w000033yK8wAAE,False,NaN,NaN,NaN,NaN
1,asdf1234@invalidaddress5.org,0035w00003DA2NpAAL,False,NaN,NaN,NaN,NaN
2,tonya@sample.com,0035w000033yK8yAAE,False,NaN,NaN,NaN,NaN
3,alex@bco.com,0035w000033yK8zAAE,False,NaN,NaN,NaN,NaN


,Email,Id,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
224742,ncn@nanohub.org,0035w000031Vsp4AAC,False,purdue university,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR,1685.0
224743,gridstat@nanohub.org,0035w000031Vsp3AAC,False,purdue university,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR,1684.0
224744,support@nanohub.org,0035w000031Vsp2AAC,False,purdue university,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR,1683.0
224745,nkissebe@gmail.com,0035w000031Vsp1AAC,False,NaN,NaN,,998.0


In [16]:
contact_org_df = contact_org_df.drop(columns='Id')

In [17]:
display(contact_org_df.head(2))

,Email,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
0,jack@test.com,False,NaN,NaN,NaN,NaN
1,asdf1234@invalidaddress5.org,False,NaN,NaN,NaN,NaN


In [18]:
## fixing commercial email hosts - zero out both Organization_composite__c and Organization_email_derived__c  
indices = [i for i, x in enumerate(user_domains.to_list()) if x == ' ']

#contact_org_df['Organization_composite__c'][indices] = ' '
contact_org_df['Organization_email_derived__c'][indices] = ' '


C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [19]:
## all_org_df - find all commercial names that correspond to a commerical email domain
nh_domains = all_org_df['Domain__c'].to_list()
nh_cdi = [] #cdi = commercial domain indexes

for i,j in enumerate(nh_domains):
    if j in cdomains.domains:
        nh_cdi.append(i)


In [20]:
nh_c_all = all_org_df.iloc[nh_cdi,:]

display(nh_c_all.head(2))
nh_c_all = nh_c_all['Id'].to_list()

,Domain__c,Id,Name
13,yahoo.com,a0r5w00000V42iXAAR,yahoo
24,aol.com,a0r5w00000V42iiAAB,aol


In [21]:
## go through every single composite organization, and ensure that they are all non-commercial
composite_org_verify = contact_org_df['Organization_composite__c'][indices].to_list()

for i,j in enumerate(composite_org_verify):
    if j in nh_c_all:
        contact_org_df.iloc[i,3] = ' '


In [22]:
display(contact_org_df.head(100))

,Email,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
0,jack@test.com,False,NaN,NaN,NaN,NaN
1,asdf1234@invalidaddress5.org,False,NaN,NaN,NaN,NaN
2,tonya@sample.com,False,NaN,NaN,NaN,NaN
3,alex@bco.com,False,NaN,NaN,NaN,NaN
4,ed@getoutdoors.com,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
95,NaN,False,NaN,NaN,,298517.0
96,cai312@purdue.edu,False,NaN,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR,298516.0
97,NaN,False,NaN,NaN,,298515.0
98,zadeel2@uic.edu,False,NaN,a0r5w00000V42idAAB,a0r5w00000V42idAAB,298514.0


## To Salesforce Sales Cloud CRM

In [23]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [24]:
# send data to Salesforce
db_s.send_data(contact_org_df)

[Success] Bulk job creation successful. Job ID = 7505w00000PVk1iAAD
hello
[Success] CSV upload successful. Job ID = 7505w00000PVk1iAAD
[Success] Closing job successful. Job ID = 7505w00000PVk1iAAD


In [30]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000PVk1iAAD',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-09-02T16:54:53.000+0000',
 'systemModstamp': '2020-09-02T17:04:10.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 224746,
 'numberRecordsFailed': 111,
 'retries': 0,
 'totalProcessingTime': 2320763,
 'apiActiveProcessingTime': 2102069,
 'apexProcessingTime': 23149609}

In [26]:
from pprint import pprint
pprint(db_s.check_bulk_failed_results())

''
